!pip install flask qrcode pillow ollama ipython-sql

In [4]:
import requests
import json
import os

# Define o caminho base para o projeto
base_path = '/home/gustavo/Detetive_Game'
os.chdir(base_path)  # Muda para o diretório do projeto

print(f"Diretório do projeto: {os.getcwd()}")

try:
    # Teste simples para ver se o Ollama está respondendo
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "llama3",
            "prompt": "Olá, tudo bem?",
            "stream": False
        }
    )
    
    if response.status_code == 200:
        print("✅ Ollama está rodando corretamente com o modelo llama3")
        print("Resposta de teste:", response.json()["response"])
    else:
        print("⚠️ Ollama está rodando, mas retornou um erro:", response.status_code)
        print(response.text)
except Exception as e:
    print("❌ Erro ao conectar com Ollama. Certifique-se que está rodando com 'ollama serve'")
    print(f"Erro: {str(e)}")

# Criar diretórios necessários
# Usar caminhos absolutos para garantir que estão no local correto
dirs_to_create = ['qrcodes', 'static', 'templates', 'Data']

for dir_name in dirs_to_create:
    dir_path = os.path.join(base_path, dir_name)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f"✅ Diretório '{dir_name}' criado em {dir_path}")
    else:
        print(f"✅ Diretório '{dir_name}' já existe em {dir_path}")

print("\n🎮 Configuração de diretórios concluída!")

# Criação do banco de dados
import sqlite3
from IPython.display import display, HTML

# Define o caminho do banco de dados usando o caminho base
db_path = os.path.join(base_path, 'Data/detetive_game.db')
print(f"O banco de dados será criado em: {db_path}")

# Função para criar o banco de dados
def criar_banco_de_dados(db_path):
    """Cria o esquema do banco de dados para o jogo de detetive"""
    # Garante que o diretório existe
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Tabela de Histórias
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS historias (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        titulo TEXT NOT NULL,
        descricao TEXT NOT NULL,
        cenario TEXT NOT NULL,
        envolvido_nome TEXT NOT NULL,
        envolvido_descricao TEXT NOT NULL,
        envolvido_background TEXT,
        tempo_ocorrido TEXT,
        tipo_misterio TEXT NOT NULL,
        detalhes_misterio TEXT,
        culpado_id TEXT,
        motivo TEXT,
        objeto_chave TEXT,
        data_criacao TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    # Tabela de personagens
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS personagens (
        id TEXT PRIMARY KEY,
        historia_id INTEGER NOT NULL,
        nome TEXT NOT NULL,
        role TEXT NOT NULL,
        personalidade TEXT NOT NULL,
        background TEXT,
        comportamento TEXT,
        conhecimento_inicial TEXT,
        estado_emocional_inicial TEXT,
        temperatura REAL DEFAULT 0.7,
        top_p REAL DEFAULT 0.9,
        FOREIGN KEY (historia_id) REFERENCES historias (id) ON DELETE CASCADE
    )
    ''')
    
    # Tabela para evolução de personagens
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS evolucao_personagem (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        personagem_id TEXT NOT NULL,
        gatilho_evento TEXT NOT NULL,
        condicao TEXT NOT NULL,
        novo_conhecimento TEXT,
        novo_estado_emocional TEXT,
        nova_atitude TEXT,
        revelacao TEXT,
        ordem_sequencia INTEGER,
        FOREIGN KEY (personagem_id) REFERENCES personagens (id) ON DELETE CASCADE
    )
    ''')
    
    # Tabela de jogadores
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS jogadores (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        identificador TEXT UNIQUE NOT NULL,
        ativo BOOLEAN DEFAULT 1,
        data_criacao TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    # Tabela para rastrear progresso individual dos jogadores
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS progresso_jogador (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        jogador_id INTEGER NOT NULL,
        historia_id INTEGER NOT NULL,
        status TEXT DEFAULT 'em_andamento',
        data_inicio TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        data_conclusao TIMESTAMP,
        posicao_ranking INTEGER,
        teoria_final TEXT,
        acertos INTEGER,
        total_pistas_encontradas INTEGER,
        tempo_solucao INTEGER, -- em segundos
        FOREIGN KEY (jogador_id) REFERENCES jogadores (id) ON DELETE CASCADE,
        FOREIGN KEY (historia_id) REFERENCES historias (id) ON DELETE CASCADE
    )
    ''')
    
    # Tabela para rastrear pistas descobertas por cada jogador
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS pistas_descobertas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        jogador_id INTEGER NOT NULL,
        pista_id TEXT NOT NULL,
        data_descoberta TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        compartilhada BOOLEAN DEFAULT 0,
        FOREIGN KEY (jogador_id) REFERENCES jogadores (id) ON DELETE CASCADE,
        FOREIGN KEY (pista_id) REFERENCES pistas (id) ON DELETE CASCADE
    )
    ''')
    
    # Tabela para rastrear interações entre jogadores e personagens
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS interacoes_personagem (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        jogador_id INTEGER NOT NULL,
        personagem_id TEXT NOT NULL,
        estagio_evolucao INTEGER DEFAULT 0,
        ultima_interacao TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        pistas_reveladas TEXT, -- Armazenado como JSON com IDs das pistas
        contexto_conversa TEXT, -- Para manter o histórico recente da conversa
        FOREIGN KEY (jogador_id) REFERENCES jogadores (id) ON DELETE CASCADE,
        FOREIGN KEY (personagem_id) REFERENCES personagens (id) ON DELETE CASCADE
    )
    ''')
    
    # Tabela para QR codes e seus significados no jogo
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS qr_codes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        codigo TEXT UNIQUE NOT NULL,
        tipo TEXT NOT NULL, -- 'personagem', 'pista', 'local', etc.
        referencia_id TEXT NOT NULL, -- ID do objeto referenciado (personagem_id, pista_id, local_id)
        descricao TEXT,
        ativo BOOLEAN DEFAULT 1,
        data_criacao TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    # Tabela de locais
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS locais (
        id TEXT PRIMARY KEY,
        historia_id INTEGER NOT NULL,
        nome TEXT NOT NULL,
        descricao TEXT NOT NULL,
        FOREIGN KEY (historia_id) REFERENCES historias (id) ON DELETE CASCADE
    )
    ''')
    
    # Tabela de pistas
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS pistas (
        id TEXT PRIMARY KEY,
        historia_id INTEGER NOT NULL,
        local_id TEXT NOT NULL,
        nome TEXT NOT NULL,
        descricao TEXT NOT NULL,
        importancia TEXT,
        oculta BOOLEAN DEFAULT 0,
        FOREIGN KEY (historia_id) REFERENCES historias (id) ON DELETE CASCADE,
        FOREIGN KEY (local_id) REFERENCES locais (id) ON DELETE CASCADE
    )
    ''')
    
    conn.commit()
    conn.close()
    
    return db_path

# Cria o banco de dados
db_file = criar_banco_de_dados(db_path)

# Verifica se o banco foi criado e verifica a estrutura
def verificar_banco(db_path):
    """Verifica se o banco foi criado corretamente e mostra as tabelas"""
    if os.path.exists(db_path):
        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()
            
            # Lista todas as tabelas no banco
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tabelas = cursor.fetchall()
            
            print(f"✅ Banco de dados criado com sucesso em: {db_path}")
            print(f"Tabelas criadas ({len(tabelas)}):")
            for tabela in tabelas:
                print(f"  - {tabela[0]}")
                
                # Mostra a estrutura de cada tabela
                cursor.execute(f"PRAGMA table_info({tabela[0]})")
                colunas = cursor.fetchall()
                print(f"    Colunas ({len(colunas)}):")
                for coluna in colunas:
                    print(f"      {coluna[1]} ({coluna[2]})")
            
            conn.close()
            return True
        except Exception as e:
            print(f"❌ Erro ao verificar o banco de dados: {str(e)}")
            return False
    else:
        print(f"❌ Banco de dados não foi encontrado em: {db_path}")
        return False

# Verifica o banco
verificar_banco(db_path)

print("\n🎮 Configuração completa! O ambiente está pronto para o desenvolvimento do jogo.")

Diretório do projeto: /home/gustavo/Detetive_Game
✅ Ollama está rodando corretamente com o modelo llama3
Resposta de teste: Olá! Sim, tudo bem, obrigado (ou ogra)! Eu sou um modelo de linguagem treinado para falar português e ajudar com perguntas e respostas. Como posso ajudar você hoje?
✅ Diretório 'qrcodes' já existe em /home/gustavo/Detetive_Game/qrcodes
✅ Diretório 'static' já existe em /home/gustavo/Detetive_Game/static
✅ Diretório 'templates' já existe em /home/gustavo/Detetive_Game/templates
✅ Diretório 'Data' já existe em /home/gustavo/Detetive_Game/Data

🎮 Configuração de diretórios concluída!
O banco de dados será criado em: /home/gustavo/Detetive_Game/Data/detetive_game.db
✅ Banco de dados criado com sucesso em: /home/gustavo/Detetive_Game/Data/detetive_game.db
Tabelas criadas (11):
  - historias
    Colunas (14):
      id (INTEGER)
      titulo (TEXT)
      descricao (TEXT)
      cenario (TEXT)
      envolvido_nome (TEXT)
      envolvido_descricao (TEXT)
      envolvido_bac

In [5]:
import sys
print(sys.executable)

/home/gustavo/py_env1/bin/python3
